In [3]:
import numpy as np
import pickle
import torch

import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], os.pardir, os.pardir))
from utils import SimpleProblem

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

torch.set_default_dtype(torch.float64)

import gurobipy as gp
from gurobipy import Model, GRB, QuadExpr, LinExpr

from utils import build_gurobi_model
import time

In [4]:
from model_utils import NNSolver_eq_proj

filepath = "/home/jxxiong/A-xjx/DC3/datasets/simple/random_simple_dataset_var200_ineq100_eq100_ex10000"
with open(filepath, 'rb') as f:
    data = pickle.load(f)

arg_path = "/home/jxxiong/A-xjx/DC3/results/SimpleProblem-200-100-100-10000/method_eq_proj/6e159980feacc6482172a8c3a901f63bc1ce625e/1712566930-9855366/args.dict"
args = pickle.load(open(arg_path, 'rb'))

model = NNSolver_eq_proj(data, args)

model_path = "/home/jxxiong/A-xjx/DC3/results/SimpleProblem-200-100-100-10000/method_eq_proj/6e159980feacc6482172a8c3a901f63bc1ce625e/1712566930-9855366/solver_net.dict"
model.load_state_dict(torch.load(model_path))

model.eval()
model.to('cuda')
# model.to('cpu')

NNSolver_eq_proj(
  (net): Sequential(
    (0): Linear(in_features=100, out_features=200, bias=True)
    (1): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=200, out_features=200, bias=True)
    (5): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.2, inplace=False)
    (8): Linear(in_features=200, out_features=200, bias=True)
  )
)

In [10]:
DEVICE = 'cuda'
for attr in dir(data):
    var = getattr(data, attr)
    if not callable(var) and not attr.startswith("__") and torch.is_tensor(var):
        try:
            setattr(data, attr, var.to(DEVICE))
        except AttributeError:
            pass

In [11]:
A = data.A
G = data.G
h = data.h
Q = data.Q
p = data.p

A_np = data.A_np
G_np = data.G_np
h_np = data.h_np
Q_np = data.Q_np
p_np = data.p_np

X = data.X
X_np = data.X_np

num_var = A.shape[1]
num_eq = A.shape[0]
num_ineq = G.shape[0]

In [12]:
idx = 1234

In [13]:
full_model = build_gurobi_model(Q_np, p_np, A_np, X_np[idx], G_np, h_np)
full_model.setParam('OutputFlag', 1)
full_model.optimize()
optimal_val_full = full_model.objVal
time_full = full_model.Runtime
optimal_x_full = np.array([v.x for v in full_model.getVars()])
duals_full = np.array([c.Pi for c in full_model.getConstrs()])
duals_ineq_target = duals_full[-G.shape[0]:]
duals_eq_target = duals_full[:A.shape[0]]
active_ineq_target = np.where(np.abs(G_np@optimal_x_full - h_np) <= 1e-4)[0]
active_ineq_target.tolist()
active_ineq_index_target = [50 + x for x in active_ineq_target]
active_ineq_index_target

Set parameter OutputFlag to value 1
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.2 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i9-12900K, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 200 rows, 200 columns and 40000 nonzeros
Model fingerprint: 0xae740817
Model has 200 quadratic objective terms
Coefficient statistics:
  Matrix range     [7e-06, 4e+00]
  Objective range  [2e-03, 1e+00]
  QObjective range [1e-02, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-02, 1e+01]
Presolve time: 0.01s
Presolved: 200 rows, 200 columns, 40000 nonzeros
Presolved model has 200 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 200
 AA' NZ     : 1.990e+04
 Factor NZ  : 2.010e+04
 Factor Ops : 2.687e+06 (less than 1 second per iteration)
 Threads    : 24

                  Objective                Residual
Iter       Primal          Dual     

[67, 69, 73, 83, 84, 87, 93, 96, 98, 102, 107, 116, 134, 136, 141, 142, 144]

# remove part of the inequality constraints

In [14]:
start_time = time.time()
Yhat = model(X[idx].unsqueeze(0).to('cuda'))
Ystar = data.projection(X[idx].unsqueeze(0).to('cuda'), Yhat)
end_time = time.time()


RuntimeError: CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)`

In [15]:
start_time = time.time()
%time Yhat = model(X[idx].unsqueeze(0))
%time Ystar = data.projection(X[idx].unsqueeze(0), Yhat)
end_time = time.time()
inference_time = end_time - start_time
print("Inference time: ", inference_time)
Ystar = Ystar.detach().numpy().flatten()
ineq_sorted_index = np.argsort(-G_np@Ystar + h_np)
ineq_sorted = np.sort(-G_np@Ystar + h_np)
print(ineq_sorted)
print(ineq_sorted_index)
inactive_ineq_index = ineq_sorted_index[ineq_sorted > 2]
print(inactive_ineq_index)

RuntimeError: CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)`

NameError: name 'Yhat' is not defined

Inference time:  0.12717771530151367


NameError: name 'Ystar' is not defined

In [ ]:
set(inactive_ineq_index).intersection(set(active_ineq_target))

set()

In [ ]:
np.setdiff1d(np.arange(num_ineq), active_ineq_target).shape

(83,)

In [ ]:
remaining_ineq_index = np.setdiff1d(np.arange(num_ineq), inactive_ineq_index)
# remaining_ineq_index = active_ineq_target[60:]


G_remaining = G[remaining_ineq_index]
h_remaining = h[remaining_ineq_index]
model_reduced = build_gurobi_model(Q, p, A, X_np[idx], G_remaining, h_remaining)
model_reduced.setParam('OutputFlag', 1)
# initialize the primal variables in model_hat2
# for i, v in enumerate(model_hat2.getVars()):
#     v.setAttr('Start', Ystar[i])
model_reduced.optimize()
optimal_val = model_reduced.objVal
time_reduced = model_reduced.Runtime + inference_time
optimal_x = np.array([v.x for v in model_reduced.getVars()])
print("Value: full={}, reduced={}, diff={}".format(optimal_val_full, optimal_val, optimal_val_full - optimal_val))
print("Time: full={}, reduced={}, diff={}".format(time_full, time_reduced, time_full - time_reduced))

Set parameter OutputFlag to value 1
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.2 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i9-12900K, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 122 rows, 200 columns and 24400 nonzeros
Model fingerprint: 0xc764a257
Model has 200 quadratic objective terms
Coefficient statistics:
  Matrix range     [7e-06, 4e+00]
  Objective range  [2e-03, 1e+00]
  QObjective range [1e-02, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-02, 9e+00]
Presolve time: 0.01s
Presolved: 122 rows, 200 columns, 24400 nonzeros
Presolved model has 200 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 200
 AA' NZ     : 7.381e+03
 Factor NZ  : 7.503e+03
 Factor Ops : 6.127e+05 (less than 1 second per iteration)
 Threads    : 24

                  Objective                Residual
Iter       Primal          Dual     

# osqp warm start

In [ ]:
from utils import build_osqp

num_trial = 10

total_time = 0.0
for _ in range(num_trial):
    solver = build_osqp(Q_np, p_np, A_np, X_np[idx], G_np, h_np)
    # solver1.warm_start(x=Ystar)
    start_time = time.time()
    results = solver.solve()
    end_time = time.time()
    total_time += (end_time - start_time)
print(total_time / num_trial)
print(results.info.obj_val)


0.004205560684204102
-29.200070154743347


In [ ]:
total_time_ws = 0.0
for _ in range(num_trial):
    solver_ws = build_osqp(Q_np, p_np, A_np, X_np[idx], G_np, h_np)
    solver_ws.warm_start(x=Ystar)
    start_time = time.time()
    result_ws = solver_ws.solve()
    end_time = time.time()
    total_time_ws += (end_time - start_time)
print(total_time_ws / num_trial + inference_time)
print(result_ws.info.obj_val)

0.004297995567321777
-29.200039984137756


In [ ]:
total_time_reduced = 0.0
for _ in range(num_trial):
    solver_reduced = build_osqp(Q_np, p_np, A_np, X_np[idx], G_remaining, h_remaining)
    start_time = time.time()
    results_reduced = solver_reduced.solve()
    end_time = time.time()
    total_time_reduced = end_time - start_time
print(total_time_reduced / num_trial + inference_time)
print(results_reduced.info.obj_val)

0.0023162126541137694
-29.200051564649282


In [ ]:
total_time_reduced_ws = 0.0
for _ in range(num_trial):
    solver_reduced_ws = build_osqp(Q_np, p_np, A_np, X_np[idx], G_remaining, h_remaining)
    solver_reduced_ws.warm_start(x=Ystar)
    start_time = time.time()
    results_reduced_ws = solver_reduced_ws.solve()
    end_time = time.time()
    total_time_reduced_ws = end_time - start_time
print(total_time_reduced_ws / num_trial + inference_time)
print(results_reduced_ws.info.obj_val)

0.0022213935852050783
-29.200125160105603


# check

In [ ]:
error = []
for i in range(9300, 10000):
    full_model = build_gurobi_model(Q_np, p_np, A_np, X_np[idx], G_np, h_np)
    full_model.optimize()
    optimal_x_full = np.array([v.x for v in full_model.getVars()])
    active_ineq_target = np.where(np.abs(G_np@optimal_x_full - h_np) <= 1e-4)[0]
    active_ineq_target.tolist()

    Yhat = model(X[idx].unsqueeze(0))
    Ystar = data.projection(X[idx].unsqueeze(0), Yhat)
    Ystar = Ystar.detach().numpy().flatten()
    ineq_sorted_index = np.argsort(-G_np@Ystar + h_np)
    ineq_sorted = np.sort(-G_np@Ystar + h_np)
    inactive_ineq_index = ineq_sorted_index[ineq_sorted > 2]

    error.append(len(set(inactive_ineq_index).intersection(set(active_ineq_target))))
print(max(error))



0
